# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
import numpy as np
import pandas as pd

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
df = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
df.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
df.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
df['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
df['Department'].value_counts()

POLICE                   13414
FIRE                      4641
STREETS & SAN             2198
OEMC                      2102
WATER MGMNT               1879
AVIATION                  1629
TRANSPORTN                1140
PUBLIC LIBRARY            1015
GENERAL SERVICES           980
FAMILY & SUPPORT           615
FINANCE                    560
HEALTH                     488
CITY COUNCIL               411
LAW                        407
BUILDINGS                  269
COMMUNITY DEVELOPMENT      207
BUSINESS AFFAIRS           171
COPA                       116
BOARD OF ELECTION          107
DoIT                        99
PROCUREMENT                 92
INSPECTOR GEN               87
MAYOR'S OFFICE              85
CITY CLERK                  84
ANIMAL CONTRL               81
HUMAN RESOURCES             79
CULTURAL AFFAIRS            65
BUDGET & MGMT               46
ADMIN HEARNG                39
DISABILITIES                28
TREASURER                   22
HUMAN RELATIONS             16
BOARD OF

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
from scipy import stats


scipy.**stats.ttest_1samp**(a, popmean, axis=0)

### Parameters:
**a** : array_like

sample observation

**popmean** : float or array_like

expected value in null hypothesis, if array_like than it must have the same shape as a excluding the axis dimension

**axis** : int, optional, (default axis=0)

Axis can equal None (ravel array first), or an integer (the axis over which to operate on a).

### Returns:
**t** : float or array

t-statistic

**prob** : float or array

two-tailed p-value


In [8]:
hourly_workers = df[df['Salary or Hourly'] == 'Hourly']
hourly_rate_mean = hourly_workers['Hourly Rate'].mean()
hourly_workers= hourly_workers['Hourly Rate']
hourly_rate_mean

32.78855771628024

In [9]:
# random.choice takes a random sample from the database
sample = np.random.choice(hourly_workers, replace=False, size=1000)
sample_mean = sample.mean()
sample_std = sample.std()

In [10]:
# OPTION 1
h0 = 30
t,prob = stats.ttest_1samp(sample,h0)
level_of_significance = 0.05


print('This would be how far from being accepted it was (I think 🤯):\n',(level_of_significance - prob )*100,'%')
print('This would be what should the confidence interval be to accept the null hypotheis, (I think again 🤯):\n',(100-(prob )*100),'%')

if(prob < level_of_significance):
    print('\nReject null hypothesis')
elif(prob>= level_of_significance):
    print('Accept null hypothesis')
else:
    print('Something went wrong')

This would be how far from being accepted it was (I think 🤯):
 5.0 %
This would be what should the confidence interval be to accept the null hypotheis, (I think again 🤯):
 100.0 %

Reject null hypothesis


In [110]:
# OPTION 2
deg_of_freedom = len(sample)-1

dist = stats.t(deg_of_freedom, 30, sample_std/np.sqrt(len(sample)))
dist_interval = dist.interval(0.95)
dist_interval

(29.26936867016147, 30.73063132983853)

In [111]:
if(not(dist_interval[0]< sample_mean) & (dist_interval[1]> sample_mean)):
    print('Reject null hypothesis')
elif(dist_interval[0]<= sample_mean) & (dist_interval[1]>= sample_mean):
    print('Do not reject null hypothesis')
else:
    print('Something went wrong')

Reject null hypothesis


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [133]:
police_df = df[df['Department'] == "POLICE"]
police_mean = police_df["Annual Salary"]. mean()

# QUES - Should I have taken a sample? Guess no 🤯
sample = np.random.choice(police_df['Annual Salary'], replace=False, size=1000)

In [146]:
# 1) Test your hypothesis -> H0 : μ > 86.000
# 2) Alternative hypothesis -> H1 : μ <= 86.000
# 3) Condidence interval 95%
# 4) Level of significance 0.25

level_of_significance =  0.025
h0 = 86000

statistic,pvalue = stats.ttest_1samp(sample,h0)
pvalue_one_tale = pvalue/2
print("p_value = pvalue/2 for one tailed test:", pvalue_one_tale)

if(pvalue_one_tale < level_of_significance):
    print('\nReject null hypothesis')
elif(pvalue_one_tale >= level_of_significance):
    print('Accept null hypothesis')
else:
    print('Something went wrong')

pvalue_one_tale
# It is sometimes accepting, sometimes rejecting and sometimes giving me an error(nan) 🤔
# I think there is something wrong here 

p_value = pvalue/2 for one tailed test: 0.1547722295332203
Accept null hypothesis


0.1547722295332203

Using the `crosstab` function, find the department that has the most hourly workers. 

In [136]:
crosstab_df = pd.crosstab(df['Department'], df['Salary or Hourly']).sort_values('Hourly',ascending=False)

hourly_department = crosstab_df.iloc[0].name
print('Department with the most hourly workers:',hourly_department)

crosstab_df.head()

Department with the most hourly workers: STREETS & SAN


Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [168]:
# h0: mean < 35
# h1: mean >= 35
hourly_workers_streets = df[(df['Department'] == hourly_department) & (df['Salary or Hourly'] == 'Hourly')]['Hourly Rate']

h0 = 35
statistic,pvalue = stats.ttest_1samp(hourly_workers_streets,h0)
level_of_significance = 0.025


print('This would be how far from being accepted it was (I think 🤯):\n',(level_of_significance - pvalue )*100,'%')
print('This would be what should the confidence interval be to accept the null hypotheis, (I think again 🤯):\n',(100-(pvalue )*100),'%')

if(pvalue < level_of_significance):
    print('\nReject null hypothesis')
elif(pvalue>= level_of_significance):
    print('Accept null hypothesis')
else:
    print('Something went wrong')
    

This would be how far from being accepted it was (I think 🤯):
 2.5 %
This would be what should the confidence interval be to accept the null hypotheis, (I think again 🤯):
 100.0 %

Reject null hypothesis


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [165]:
hourly_workers = df[df['Salary or Hourly'] == 'Hourly']['Hourly Rate']
hourly_workers

4        19.86
6        46.10
7        35.60
10        2.65
18       17.68
         ...  
33164    46.10
33168    17.68
33169    35.60
33174    46.35
33175    48.85
Name: Hourly Rate, Length: 8022, dtype: float64

In [181]:

confidence_level = 0.95
degrees_of_freedom = len(hourly_workers) -1
mean = hourly_workers.mean()
std = hourly_workers.std()
# Option 1 for standard error
standard_error = std/np.sqrt(len(hourly_workers))
# Option 2 for standard error
standard_error = stats.sem(hourly_workers)


dist = stats.t(deg_of_freedom, 30, standard_error)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [182]:
# Your code here:

dist_interval = dist.interval(confidence_level)
dist_interval

(29.73461910939676, 30.26538089060324)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

